# Libraries

In [1]:
%pip install -q -U datasets transformers accelerate sentencepiece

In [2]:
import os
import random
import numpy as np
import torch
import transformers
from pprint import pprint
from datetime import datetime
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding, Trainer, TrainingArguments
from sklearn.metrics import f1_score

# Config

In [3]:
seed = 42
lang = 'sun'

hf_model_avg_id = 'FacebookAI/roberta-base'
hf_model_ids = [
    'alxxtexxr/RoBERTa-Base-SE2025T11A-sun-v20241222031929',
    'alxxtexxr/RoBERTa-Base-SE2025T11A-sun-v20241222033713',
    'alxxtexxr/RoBERTa-Base-SE2025T11A-sun-v20241222034933',
    'alxxtexxr/RoBERTa-Base-SE2025T11A-sun-v20241222041052',
]
hf_tokenizer_id = hf_model_avg_id
hf_data_id = 'alxxtexxr/SemEval2025-Task11-Dataset'
hf_data_config = 'track_a_sun_70_15_15_stratify_v2'

# Init. averaging weights based on model F1 scores
model_f1s = [0.671053, 0.711409, 0.649351, 0.716216]
model_rank = [sorted(model_f1s).index(x) + 1 for x in model_f1s]
# avg_weights = [w / sum(model_f1s) for w in model_f1s]
avg_weights = [w / sum(model_rank) for w in model_rank]
assert len(avg_weights) == len(hf_model_ids)
assert sum(avg_weights) == 1
print("Model F1 scores:", model_f1s)
print("Averaging weights:", avg_weights)

project_name = f'RoBERTa-Base-WAvg-SE2025T11A-{lang}-v{datetime.now().strftime("%Y%m%d%H%M%S")}'

Model F1 scores: [0.671053, 0.711409, 0.649351, 0.716216]
Averaging weights: [0.2, 0.3, 0.1, 0.4]


In [4]:
def set_seed(seed):
    # Set random seed for NumPy
    np.random.seed(seed)

    # Set random seed for Torch
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if using multi-GPU
    torch.backends.cudnn.deterministic = True  # Ensures deterministic results
    torch.backends.cudnn.benchmark = False  # Avoids non-deterministic algorithms

    # Set random seed for Transformers
    transformers.set_seed(seed)

    # Optionally set random seed for sklearn and Python's own random module
    random.seed(seed)

    # Set random seed for os
    os.environ['PYTHONHASHSEED'] = str(seed)

    print(f"Random seed set to: {seed}")

set_seed(seed)

Random seed set to: 42


# Data

## Load Data

In [5]:
datasets = load_dataset(hf_data_id, hf_data_config)

cols = list(datasets['train'].features)
emotion_cols = [col for col in cols if col not in ['Unnamed: 0', 'text', 'emotion']]
splits = [*datasets.keys()]

print("Data columns:", cols)
print("Emotions columns:", emotion_cols)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


Data columns: ['text', 'emotion', 'marah', 'jijik', 'takut', 'senang', 'sedih', 'terkejut', 'biasa']
Emotions columns: ['marah', 'jijik', 'takut', 'senang', 'sedih', 'terkejut', 'biasa']


In [6]:
class2id = {class_:id for id, class_ in enumerate(emotion_cols)}
id2class = {id:class_ for class_, id in class2id.items()}

print("Class to ID:\n")
pprint(class2id, width=1)
print()
print("ID to Class:\n")
pprint(id2class, width=1)

Class to ID:

{'biasa': 6,
 'jijik': 1,
 'marah': 0,
 'sedih': 4,
 'senang': 3,
 'takut': 2,
 'terkejut': 5}

ID to Class:

{0: 'marah',
 1: 'jijik',
 2: 'takut',
 3: 'senang',
 4: 'sedih',
 5: 'terkejut',
 6: 'biasa'}


## Preprocess Data

In [7]:
tokenizer = AutoTokenizer.from_pretrained(hf_tokenizer_id)

In [8]:
def one_hot_encode_emotion(emotion, emotion_cols):
    emotions = emotion.replace(" ", "").split(",")
    one_hot_emotion = [1.0 if emotion_col in emotions else 0.0 for emotion_col in emotion_cols] # Ensure that the label is float, not int
    return one_hot_emotion

def preprocess_function(data):
   text = data['text']
   emotion = data['emotion']
   labels = one_hot_encode_emotion(emotion, emotion_cols)
   data = tokenizer(text, truncation=True)
   data['labels'] = labels
   return data

tokenized_datasets = {split: datasets[split].map(preprocess_function) for split in splits}

Map:   0%|          | 0/128 [00:00<?, ? examples/s]

In [9]:
# Sanity check
data = tokenized_datasets['train'][5]

print("Text:", data['text'])
print("Emotion(s):", data['emotion'])
print("Labels:", data['labels'], '-->', emotion_cols)

Text: ☝️Cung urang Kuningan, Pas urang waduk Darma, Kpan atuh main ka waduk Darma Dai, ??? Di antos, Aa sareng tteh Na, 🤗🤗🤗
Emotion(s): senang
Labels: [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0] --> ['marah', 'jijik', 'takut', 'senang', 'sedih', 'terkejut', 'biasa']


In [10]:
data_collator = DataCollatorWithPadding(tokenizer)

# Model

In [11]:
models = []
for hf_model_id in hf_model_ids:
    model = AutoModelForSequenceClassification.from_pretrained(
        hf_model_id, 
        num_labels=len(emotion_cols),
        id2label=id2class, 
        label2id=class2id,
        problem_type = "multi_label_classification",
    )
    models.append(model)

# print("Models:", models)

In [12]:
model_avg = AutoModelForSequenceClassification.from_pretrained(
    hf_model_avg_id, 
    num_labels=len(emotion_cols),
    id2label=id2class, 
    label2id=class2id,
    problem_type = "multi_label_classification",
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Model Averaging

In [13]:
with torch.no_grad():
    for param_avg in model_avg.state_dict().keys():
        weighted_sum = sum([model.state_dict()[param_avg] * w for model, w in zip(models, avg_weights)])
        model_avg.state_dict()[param_avg].copy_(weighted_sum)

# Evaluation

In [14]:
def sigmoid(x):
   return 1/(1 + np.exp(-x))

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    probs = sigmoid(predictions)
    y_pred = (probs > 0.5).astype(int)
    y_true = labels.astype(int)

    # Compute F1 score for each type of averaging method
    f1_micro = f1_score(y_true, y_pred, average='micro', zero_division=0.0)
    f1_macro = f1_score(y_true, y_pred, average='macro', zero_division=0.0)
    # f1_weighted = f1_score(y_true, y_pred, average='weighted', zero_division=0.0)
    # f1_samples = f1_score(y_true, y_pred, average='samples', zero_division=0.0)
    f1_labels = f1_score(y_true, y_pred, average=None, zero_division=0.0)
    f1_labels_dict = {f'f1_label_{emotion_cols[i]}': f1_labels[i] for i in range(len(f1_labels))}

    return {
        'f1_micro': f1_micro,
        'f1_macro': f1_macro,
        **f1_labels_dict,
    }

In [15]:
train_args = TrainingArguments(
    # Training config
    per_device_train_batch_size=2,
    num_train_epochs=8,
    learning_rate=2e-5,
    weight_decay=0.01,

    # Logging config for training
    logging_strategy='steps',
    logging_steps=100,

    # Evaluation config during training
    per_device_eval_batch_size=2,
    eval_strategy='steps',
    eval_steps=100,

    # Model saving config
    output_dir=project_name,
    save_strategy='epoch',
    # load_best_model_at_end=True,
)

trainer = Trainer(
    model=model_avg,
    args=train_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['val'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

<ipython-input-15-66cc8af75179>:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [16]:
eval = trainer.evaluate(eval_dataset=tokenized_datasets['test'])

import pandas as pd
pd.DataFrame(eval, index=[0])

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: alimtegar. Use `wandb login --relogin` to force relogin


,eval_loss,eval_model_preparation_time,eval_f1_micro,eval_f1_macro,eval_f1_label_marah,eval_f1_label_jijik,eval_f1_label_takut,eval_f1_label_senang,eval_f1_label_sedih,eval_f1_label_terkejut,eval_f1_label_biasa,eval_runtime,eval_samples_per_second,eval_steps_per_second
0,0.269037,0.0032,0.724252,0.231964,0.0,0.0,0.0,0.884615,0.73913,0.0,0.0,1.3053,98.06,49.03


In [20]:
trainer.push_to_hub(project_name)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

events.out.tfevents.1734843628.27321939cde1.11393.0:   0%|          | 0.00/847 [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.37k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/alxxtexxr/RoBERTa-Base-Avg-SE2025T11A-sun-v20241222045535/commit/036ce6dc34fa4af8bc2d45cc1570f4715cc713d3', commit_message='RoBERTa-Base-Avg-SE2025T11A-sun-v20241222045535', commit_description='', oid='036ce6dc34fa4af8bc2d45cc1570f4715cc713d3', pr_url=None, repo_url=RepoUrl('https://huggingface.co/alxxtexxr/RoBERTa-Base-Avg-SE2025T11A-sun-v20241222045535', endpoint='https://huggingface.co', repo_type='model', repo_id='alxxtexxr/RoBERTa-Base-Avg-SE2025T11A-sun-v20241222045535'), pr_revision=None, pr_num=None)

# Inference

In [17]:
data = datasets['val'][1]
text = data['text']
emotion_true = data['emotion']

inputs = tokenizer(text, return_tensors='pt').to(model_avg.device)

outputs = trainer.model(**inputs)
logits = outputs.logits
probs = sigmoid(logits.squeeze().detach().cpu().numpy()) # apply sigmoid + threshold
labels_pred = (probs > 0.5).astype(int)
emotion_pred = [id2class[idx] for idx, label in enumerate(labels_pred) if label == 1.0] # turn predicted id's into actual label names

print("Text:", text)
print("True emotion(s):", emotion_true)
print("Predicted emotion(s):", ", ".join(emotion_pred))

Text: Nongton iklan nepi ka tamat ,😭
True emotion(s): senang
Predicted emotion(s): sedih
